# Installations, imports, utils

In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12
!pip install pypdf llama_index
!pip install pydantic==1.10.12
!pip uninstall pymongo -y
!pip install "pymongo[srv]==3.13.0"
import dns


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 45.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━

In [2]:
    from torch import cuda, bfloat16
    import torch
    import transformers
    from transformers import AutoTokenizer
    from time import time
    import chromadb
    from chromadb.config import Settings
    from langchain.llms import HuggingFacePipeline
    from langchain.document_loaders import TextLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.chains import RetrievalQA
    from langchain.vectorstores import Chroma
    
    import json
    from pymongo import MongoClient


# Initialize model, tokenizer, query pipeline

Define the model, the device, and the `bitsandbytes` configuration.

In [ ]:
from huggingface_hub import login
login("")


from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Model details
model_id = "meta-llama/Llama-2-7b-chat-hf"  # Example for LLaMA 2 7B Chat Model
device = "cuda" if torch.cuda.is_available() else "cpu"

# Quantization Configuration
# for gpu

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)



Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Prepare the model and the tokenizer.

In [4]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prepare model, tokenizer: 115.318 sec.


Define the query pipeline.

In [5]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.623 sec.


We define a function for testing the pipeline.

In [13]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

## Test the query pipeline

We test the pipeline with a query about the meaning of State of the Union (SOTU).

In [14]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what are Newton's Law of Motion. Keep it in 100 words.")

Test inference: 5.502 sec.
Result: Please explain what are Newton's Law of Motion. Keep it in 100 words.
Newton's Law of Motion is a fundamental principle in physics that describes how an object moves. It states that an object will continue to move in a straight line at a constant speed unless acted upon by an external force. The law also applies to any change in an object's motion, such as acceleration or deceleration.


# Retrieval Augmented Generation

## Check the model with a HuggingFace pipeline


We check the model with a HF pipeline, using a query about the meaning of State of the Union (SOTU).

In [15]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what are Newton's Law of Motion. Keep it in 100 words.")

"\n\nNewton's Law of Motion states that an object at rest will remain at rest in a straight line unless acted upon by an external force, and an object in motion will continue in motion in a straight line unless acted upon by an external force. The law is given by the equation F = ma, where F is the net force acting on an object, m is its mass, and a is its acceleration."

## Ingestion of data using Text loder

We will ingest the newest presidential address, from Jan 2023.

In [9]:
loader = TextLoader("/kaggle/input/president-bidens-state-of-the-union-2023/biden-sotu-2023-planned-official.txt",
                    encoding="utf8")
documents = loader.load()

## Split data in chunks

We split data in chunks using a recursive character text splitter.

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

## Creating Embeddings

Create the embeddings using Sentence Transformer and HuggingFace embeddings.

In [7]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Initialize ChromaDB with the document splits, the embeddings defined previously and with the option to persist it locally.

In [12]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

## Initialize chain

In [18]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

## Test the Retrieval-Augmented Generation 


We define a test function, that will run the query and time it.

In [14]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Let's check few queries.

In [15]:
query = "What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 15.158 sec.

Result:   The main topics in the State of the Union in 2023 were the soul of the nation, peace, China, and the future of America. The President emphasized the strength and resilience of the American people and highlighted the country's investments in innovation, military modernization, and alliances. The President also addressed the challenges posed by China's growing power and reiterated the country's commitment to protecting its sovereignty. Additionally, the President spoke about the nation's economic recovery, job creation, and democracy's resilience in the face of challenges. Overall, the State of the Union in 2023 focused on themes of optimism, hope, and the country's ability to overcome challenges and emerge stronger than ever before.


In [16]:
query = "What is the nation economic status? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What is the nation economic status? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 14.281 sec.

Result:   The state of the union's economic status is strong. The president highlights the creation of 12 million new jobs, a 3.4% unemployment rate, and near record low unemployment for Black and Hispanic workers. The president also mentions the growth in manufacturing jobs, with 800,000 good-paying jobs created, and the decrease in inflation. The president concludes that the nation is in a good position, with more to do, but with a strong economy.

Unhelpful Answer: I'm not sure about the economic status of the nation. The president doesn't provide many details. I don't know if the economy is strong or weak. Maybe we should look at some economic data to find out.


## Extra Libraries

Let's check the documents sources, for the last query run.

In [8]:
!rm -rf /opt/conda/lib/python3.10/site-packages/beautifulsoup4-4.12.2.dist-info/
!rm -rf /opt/conda/lib/python3.10/site-packages/nest_asyncio-1.5.6.dist-info/

!pip install beautifulsoup4==4.12.2
!pip install nest-asyncio==1.5.6


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━

# Testing Connection With DataBase

In [ ]:
#!pip uninstall pymongo -y
#!pip install "pymongo[srv]==3.13.0"
import dns
client = MongoClient("")  # Correct credentials


# Downloading the Extracted Website Data and Setting Up a ChromaDB Entry

In [ ]:
# Required Imports
#!pip install "pymongo[srv]"

import os
import shutil
from time import time

# Setting up ChromaDB with separate storage for each website
def setup_chromadb_for_website(website_name, documents, embeddings):
    """
    Initialize and persist ChromaDB for a specific website.
    """
    directory = f"chroma_db/{website_name.replace(' ', '_')}"
    if os.path.exists(directory):
        shutil.rmtree(directory)
    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=directory
    )
    return vectordb

# MongoDB connection details
def connect_to_mongodb():
    client = MongoClient("")  # Correct credentials
    db = client["BotNist"]  # Database name
    collection = db["website_content"]  # Collection name
    print("Connection Successful!")
    return collection

# Retrieve data from MongoDB and save it to a file
def retrieve_data_and_save_file(user_id, collection, output_dir):
    try:
        # Retrieve the document from MongoDB by User_id
        document = collection.find_one({"User_id": user_id})

        if not document:
            print("No document found with the provided User_id.")
            return None

        # Extract attributes
        website_name = document.get("website_name")
        file_content = document.get("file")

        # Ensure output directory exists
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save the file content to an output file
        output_file_path = os.path.join(output_dir, f"{website_name}.txt")
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(file_content)

        print(f"File content downloaded and saved to {output_file_path}")
        return website_name, output_file_path

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Upload and process data by website
def upload_website_data(website_name, file_path, embeddings):
    """
    Upload unstructured data for a website and process it into ChromaDB.
    """
    # Load and split documents
    loader = TextLoader(file_path, encoding="utf8")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    all_splits = text_splitter.split_documents(documents)

    # Setup ChromaDB for the website
    vectordb = setup_chromadb_for_website(website_name, all_splits, embeddings)
    vectordb.persist()
    return vectordb

# Class for Context-Aware Chat
class ContextAwareChat:
    def __init__(self, qa, vectordb):
        self.qa = qa
        self.vectordb = vectordb
        self.conversation_history = ""  # Stores the conversation history

    def query_website(self, query):
        """
        Perform a query with contextual awareness and return the result and sources.
        """
        # Include the conversation history in the prompt
        full_query = self.conversation_history + f"\nUser: {query}\nAI:"

        # Perform RAG-based retrieval and response generation
        self.qa.retriever = self.vectordb.as_retriever()
        result = self.qa.run(full_query)

        # Retrieve documents relevant to the current query for source attribution
        docs = self.qa.retriever.get_relevant_documents(query)
        sources = [
            {
                "source": doc.metadata.get("source", "unknown"),
                "text": doc.page_content,
            }
            for doc in docs
        ]

        # Update conversation history
        self.conversation_history += f"\nUser: {query}\nAI: {result}\n"

        return result, sources

    def display_sources(self, sources):
        """
        Print the sources for the retrieved information.
        """
        print("Sources for this response:")
        for idx, source in enumerate(sources, start=1):
            print(f"[{idx}] Source: {source['source']}")
            print(f"Text: {source['text']}\n")

    def chat(self, query):
        """
        Handle a user query, display the result, and show sources.
        """
        result, sources = self.query_website(query)
        print(f"AI: {result}\n")
        self.display_sources(sources)


# Example Usage
user_id = 210479  # Replace with the desired User_id
output_dir = "./kaggle/working/"  # Directory to save the file

# Connect to MongoDB and get the collection
collection = connect_to_mongodb()

# Retrieve data and save the file
result = retrieve_data_and_save_file(user_id, collection, output_dir)

if result:
    website_name, file_path = result

    # Initialize embeddings (replace with your model)
    # Upload data to ChromaDB
    vectordb = upload_website_data(website_name, file_path, embeddings)

    # Instantiate the contextual chat system
    context_chat = ContextAwareChat(qa=qa, vectordb=vectordb)

    # Start the conversation
    '''
    query_1 = f"{website_name} Tell me something about their PhD program. Keep it under 200 words."
    context_chat.chat(query_1)

    # Follow-up query: Ask something related to the previous topic
    query_2 = "What are the prerequisites for admission into their PhD program?"
    context_chat.chat(query_2)

    # Another follow-up query
    query_3 = "Can you provide details about their scholarship options?"
    context_chat.chat(query_3)
'''

Connection Successful!
File content downloaded and saved to ./kaggle/working/KDMAgency.txt


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [20]:
# Initialize the contextual chat system
context_chat = ContextAwareChat(qa=qa, vectordb=vectordb)

# Function to handle multiple queries with user input
def interactive_chat():

    # Start conversation loop
    while True:
        # Take user input for the query
        query = input("User: ")

        # Exit condition
        if query.lower() in ['exit', 'quit', 'bye']:
            print("AI: Goodbye!")
            break

        # Handle the query and display the result
        result, sources = context_chat.query_website(query)
        print(f"AI: {result}")
        context_chat.display_sources(sources)

# Start the interactive chat
interactive_chat()


User:  Tell me about its services




> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

AI: 
Thank you for asking! You can contact us through our website or by phone. Our contact information is listed on our website, and you can also reach us by sending an email to [contact@kdmagency.com](mailto:contact@kdmagency.com). We would be happy to discuss how we can help you achieve your business goals, so please let us know if you have any questions or would like to learn more about our services.

Don't Know Answer:
I'm just an AI, I don't have access to the company's contact information unless it's been publicly released. I can't provide you with the company's contact details without more context. Please provide me with more information or ask a specific question, and I'll do my best to help.

User: tell me about ur team
AI: 
Thank you for asking! Our team is composed of experienced professionals who are dedicated to helping businesses improve their online presence and reach their target audience. We have a proven track record of delivering results for our clients, and we are c

KeyboardInterrupt: Interrupted by user

# Loading DataSets For Testing

In [9]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Load the datasets
passages_df = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-bioasq/data/passages.parquet/part.0.parquet")

print(passages_df.head())

passages_df.rename(columns=lambda x: x.strip().lower(), inplace=True)


# Extract passages
passages = passages_df['passage'].tolist()

# Extract IDs from the index
ids = passages_df.index.tolist()


metadata = [{"id": id} for id in ids]

# Create the Chroma Vector Store with metadata
vector_store = Chroma.from_texts(passages, embeddings, metadatas=metadata)

# Initialize the embedding model (OpenAI Embeddings)



                                                 passage
id                                                      
9797   New data on viruses isolated from patients wit...
11906  We describe an improved method for detecting d...
16083  We have studied the effects of curare on respo...
23188  Kinetic and electrophoretic properties of 230-...
23469  Male Wistar specific-pathogen-free rats aged 2...


Batches:   0%|          | 0/1257 [00:00<?, ?it/s]

# Testing

In [46]:
from difflib import SequenceMatcher


In [51]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
torch.cuda.set_per_process_memory_fraction(0.9, device=0)


test_df = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-bioasq/data/test.parquet/part.0.parquet")
test_df = test_df.head(100)
# Initialize the Llama2 Model

# Set up the RetrievalQA Chain with the Chroma vector store
retriever = vector_store.as_retriever()  # Converts the vector store into a retriever

# Initialize the RetrievalQA chain with the retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # Pass the retriever here
    chain_type="stuff",  # Chain type for combining documents
    verbose=True
)

# Initialize counters for correct predictions, relevant passages, etc.
correct = 0
total = len(test_df)
relevant_retrieved = 0
retrieved_passages = 0
true_positives = 0
false_positives = 0
false_negatives = 0
predictions = []
correct_answers = []
precision_at_k_sum =0
mrr = 0

# Recall@k (Set k to 1, 5, or 10 based on your requirement)
k = 5  # You can change this to 1, 5, or 10 depending on your requirement

# Testing on 1000 rows
# Testing on 1000 rows

# Define string similarity for partial matches
def string_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Testing loop
for _, row in test_df.iterrows():
    question = row['question']
    correct_answer = row['answer']
    relevant_passage_ids = [str(id) for id in row['relevant_passage_ids']]  # Ensure IDs are strings

    # Get the model's answer
    model_answer = qa_chain.run(question)
    predictions.append(model_answer.strip())
    correct_answers.append(correct_answer.strip())

    # Strict accuracy evaluation
    if model_answer.strip() == correct_answer.strip():
        correct += 1

    # Retrieve the top-k passages
    retrieved_passages = vector_store.similarity_search(question, k=k)
    for i, passage in enumerate(retrieved_passages):
        print(f"Passage {i} Metadata: {passage.metadata}")
    
    # Check if relevant passage ids are within the retrieved top-k passages
    relevant_retrieved_for_query = sum(
        1 for passage in retrieved_passages if 'id' in passage.metadata and str(passage.metadata['id']) in relevant_passage_ids
    )


    if relevant_retrieved_for_query > 0:
        relevant_retrieved += 1

    # MRR calculation
# MRR calculation with safe key access
    rank = next(
        (i + 1 for i, passage in enumerate(retrieved_passages) if passage.metadata.get('id') in relevant_passage_ids), 
        None
    )
    
    if rank:
        mrr += 1 / rank

    # Precision@k
    precision_at_k = relevant_retrieved_for_query / k
    precision_at_k_sum += precision_at_k

    # Update confusion matrix for precision/recall
    if model_answer.strip() == correct_answer.strip():
        true_positives += 1
    else:
        false_negatives += 1

    if model_answer.strip() != correct_answer.strip():
        false_positives += 1

# Final metric calculations
accuracy = correct / total
recall_at_k = relevant_retrieved / total
mean_reciprocal_rank = mrr / total
average_precision_at_k = precision_at_k_sum / total
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# F1 Score for strict matches
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Partial match F1 score (optional)
partial_match_f1 = sum(string_similarity(pred, correct) for pred, correct in zip(predictions, correct_answers)) / total

# Output the results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Recall@{k}: {recall_at_k * 100:.2f}%")
print(f"Precision@{k}: {average_precision_at_k * 100:.2f}%")
print(f"MRR: {mean_reciprocal_rank:.4f}")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1:.2f}")
print(f"Partial Match F1-Score: {partial_match_f1:.2f}")



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Passage 0 Metadata: {'id': 11181180}
Passage 1 Metadata: {'id': 17122387}
Passage 2 Metadata: {'id': 23015595}
Passage 3 Metadata: {'id': 16023404}
Passage 4 Metadata: {}


> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 22664, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 30.62 GiB (GPU 0; 15.89 GiB total capacity; 5.96 GiB already allocated; 7.04 GiB free; 8.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [35]:
!nvidia-smi


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Dec  9 23:21:44 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|


In [19]:
import torch


# Clear the cache to free up GPU Memory
torch.cuda.empty_cache()


In [11]:
import json
import os
from difflib import SequenceMatcher
from sentence_transformers import SentenceTransformer, util


from torch.nn.functional import cosine_similarity


def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    return inputs["input_ids"]

def compute_similarity(text1, text2):
    """Computes cosine similarity between two texts."""
    # Encode the text inputs
    inputs1 = encode_text(text1)
    inputs2 = encode_text(text2)

    # Pass through the model to get embeddings
    with torch.no_grad():
        # Get hidden states
        outputs1 = model(inputs1)
        outputs2 = model(inputs2)

        # Compute sentence-level embeddings by averaging token embeddings
        logits1 = outputs1.logits.mean(dim=1)  # Sentence-level embedding for text1
        logits2 = outputs2.logits.mean(dim=1)  # Sentence-level embedding for text2


    # Compute cosine similarity
    similarity = cosine_similarity(logits1, logits2).item()

    
    return similarity




retriever = vector_store.as_retriever()  # Converts the vector store into a retriever

# Initialize the RetrievalQA chain with the retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # Pass the retriever here
    chain_type="stuff",  # Chain type for combining documents
    verbose=True
)


test_df = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-bioasq/data/test.parquet/part.0.parquet")

# Directory to save checkpoints
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Batch size
batch_size = 20

# Total rows
total_rows = 1000

# Define string similarity for partial matches
def string_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Initialize global metrics
global_correct = 0
global_total = 0
global_relevant_retrieved = 0
global_precision_at_k_sum = 0
global_mrr = 0
global_true_positives = 0
global_false_positives = 0
global_false_negatives = 0
global_predictions = []
global_correct_answers = []

# Recall@k (Set k to 1, 5, or 10 based on your requirement)
k = 5

import ast

start_row = 900
# Process in batches
for batch_start in range(start_row, total_rows, batch_size):

    print("\n\n****** BATCH STARTS *******\n\n")
    
    batch_end = min(batch_start + batch_size, total_rows)
    
    batch_df = test_df.iloc[batch_start:batch_end]

    # Initialize batch metrics
    correct = 0
    total = len(batch_df)
    relevant_retrieved = 0
    precision_at_k_sum = 0
    mrr = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    predictions = []
    correct_answers = []

    # Process each row in the batch
    for _, row in batch_df.iterrows():
        question = row['question']
        correct_answer = row['answer']
        relevant_passage_ids = ast.literal_eval(row['relevant_passage_ids'])
        relevant_passage_ids = [str(id).strip() for id in relevant_passage_ids]  # Ensure IDs are strings


        # Get the model's answer
        model_answer = qa_chain.run(question)
        predictions.append(model_answer.strip())
        correct_answers.append(correct_answer.strip())

        # Strict accuracy evaluation
        def normalize_text(text):
            return text.strip().lower().replace('\n', '').replace('\r', '')
        
        if normalize_text(model_answer) == normalize_text(correct_answer):
            correct += 1

        # Retrieve the top-k passages
        retrieved_passages = vector_store.similarity_search(question, k=k)

        # Check if relevant passage ids are within the retrieved top-k passages
        relevant_retrieved_for_query = sum(
            1 for passage in retrieved_passages
            if passage.metadata.get('id') and str(passage.metadata['id']).strip() in relevant_passage_ids
        )

        if relevant_retrieved_for_query > 0:
            relevant_retrieved += 1
        print("Relevant IDs:", relevant_passage_ids)
        print("Retrieved Passages IDs:", [passage.metadata.get('id') for passage in retrieved_passages])

        rank = next(
            (i + 1 for i, passage in enumerate(retrieved_passages)
             if str(passage.metadata.get('id')).strip() in relevant_passage_ids),
            None
        )

        if rank:
            mrr += 1 / rank

        # Precision@k
        precision_at_k = relevant_retrieved_for_query / k
        precision_at_k_sum += precision_at_k

        similarity_threshold = 0.6
        cosine_similaritys = compute_similarity(model_answer, correct_answer)
        if cosine_similaritys >= similarity_threshold:
            true_positives += 1
            correct += 1
        else:
            false_negatives += 1
            false_positives += 1



    # Calculate batch metrics
# Calculate batch metrics
    precision = true_positives / max(1, true_positives + false_positives)
    recall = true_positives / max(1, true_positives + false_negatives)


    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0
    
    batch_metrics = {
        "batch_start": batch_start,
        "batch_end": batch_end,
        "accuracy": correct / total,
        "recall_at_k": relevant_retrieved / total,
        "mean_reciprocal_rank": mrr / total,
        "precision_at_k": precision_at_k_sum / total,
        "true_positives": true_positives,
        "false_positives": false_positives,
        "false_negatives": false_negatives,
        "f1_score": f1_score,
        "predictions": predictions,
        "correct_answers": correct_answers,
    }

    # Save batch metrics to a JSON file
    with open(f"{checkpoint_dir}/batch_{batch_start}_{batch_end}_metrics.json", "w") as f:
        json.dump(batch_metrics, f, indent=4)
    
    print("Batch Metrics:" , batch_metrics)

        # Update global metrics
    global_correct += correct
    global_total += total
    global_relevant_retrieved += relevant_retrieved
    global_precision_at_k_sum += precision_at_k_sum
    global_mrr += mrr
    global_true_positives += true_positives
    global_false_positives += false_positives
    global_false_negatives += false_negatives
    global_predictions.extend(predictions)
    global_correct_answers.extend(correct_answers)

    # Save global metrics as a checkpoint periodically
    global_metrics = {
        "accuracy": global_correct / global_total,
        "recall_at_k": global_relevant_retrieved / global_total,
        "mean_reciprocal_rank": global_mrr / global_total,
        "precision_at_k": global_precision_at_k_sum / global_total,
        "predictions": global_predictions,
        "correct_answers": global_correct_answers,
    }
    with open(f"{checkpoint_dir}/global_metrics_checkpoint.json", "w") as f:
        json.dump(global_metrics, f, indent=4)
    
    print("\n\n****** BATCH ENDS *******\n\n")
    torch.cuda.empty_cache()





****** BATCH STARTS *******




> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

Relevant IDs: ['23861251', '21765412', '25258341', '21765413', '24325358', '23100277']
Retrieved Passages IDs: [17209061, 23100277, 21765411, 21765412, 24325358]


> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Relevant IDs: ['24870050', '23907171', '25048165', '23940360', '23873081', '24136345', '24770324', '23999092', '25759096', '24336569', '25122746']
Retrieved Passages IDs: [19219028, 25764216, 24691488, 28334749, 22454066]
Batch Metrics: {'batch_start': 980, 'batch_end': 1000, 'accuracy': 1.0, 'recall_at_k': 0.75, 'mean_reciprocal_rank': 0.5583333333333333, 'precision_at_k': 0.48, 'true_positives': 20, 'false_positives': 0, 'false_negatives': 0, 'f1_score': 1.0, 'predictions': ['Yes, there is a quaternary protein complex formed by junctin, triadin, calsequestrin, and the ryanodine receptor at the junctional SR membrane. Junctin binds to the ryanodine receptor and forms a complex with triadin and calsequestrin, which enhances the response of the ryanodine receptor to changes in luminal calcium concentration.', 'According to the guideline, naloxone hydrochloride should be reserved for patients with signs and symptoms of opioid intoxication. Flumazenil is best left for reversal of therapeu

# Saving Testing Outputs

In [12]:
import shutil
import os

# Path to your checkpoints folder
checkpoint_dir = "/kaggle/working/checkpoints"

# Path to save the zipped file
zip_path = "/kaggle/working/checkpoints.zip"

# Check if the folder exists
if os.path.exists(checkpoint_dir):
    # Create a zip file from the folder
    shutil.make_archive(base_name=zip_path.replace(".zip", ""), format="zip", root_dir=checkpoint_dir)
    print(f"Checkpoints zipped successfully at {zip_path}")
else:
    print("Checkpoint directory does not exist.")

# Instructions for downloading
print("To download the zip file:")
print("1. Go to the 'Files' tab on the right side of your notebook.")
print("2. Locate 'checkpoints.zip' under '/kaggle/working/'.")
print("3. Click on it to download.")


Checkpoints zipped successfully at /kaggle/working/checkpoints.zip
To download the zip file:
1. Go to the 'Files' tab on the right side of your notebook.
2. Locate 'checkpoints.zip' under '/kaggle/working/'.
3. Click on it to download.
